In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
# import seaborn as sn
import os
from sklearn.preprocessing import OneHotEncoder
import nltk

# Extracting recipes from csv

In [2]:
raw_interactions = pd.read_csv('./Dataset/RAW_interactions.csv')
raw_recipes = pd.read_csv('./Dataset/RAW_recipes.csv')

In [3]:
print(raw_interactions.shape)
raw_interactions.head()

(1132367, 5)


,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [4]:
print(raw_recipes.shape)
raw_recipes.head()

(231637, 12)


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [5]:
# merge recipes and interactions tables

dfinal = raw_interactions.merge(raw_recipes, how='inner', left_on='recipe_id', right_on='id')
print(dfinal.shape)
dfinal.head()

(1132367, 17)


,user_id,recipe_id,date,rating,review,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...,white bean green chile pepper soup,40893,495,1533,2002-09-21,"['weeknight', 'time-to-make', 'course', 'main-...","[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]",4,"['combine beans , onion , chilies , 1 / 2 teas...",easy soup for the crockpot.,"['great northern beans', 'yellow onion', 'dice...",9
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall...",white bean green chile pepper soup,40893,495,1533,2002-09-21,"['weeknight', 'time-to-make', 'course', 'main-...","[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]",4,"['combine beans , onion , chilies , 1 / 2 teas...",easy soup for the crockpot.,"['great northern beans', 'yellow onion', 'dice...",9
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...,devilicious cookie cake delights,44394,20,56824,2002-10-27,"['30-minutes-or-less', 'time-to-make', 'course...","[132.3, 11.0, 39.0, 5.0, 4.0, 11.0, 5.0]",5,"['blend together cake mix , oil and eggs', 'ad...",NaN,"[""devil's food cake mix"", 'vegetable oil', 'eg...",4
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...,baked potato toppings,85009,10,64342,2004-02-25,"['15-minutes-or-less', 'time-to-make', 'course...","[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]",3,['pick whichever topping you want to use and c...,these toppings sure makes a nice change from p...,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...",13
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin...",baked potato toppings,85009,10,64342,2004-02-25,"['15-minutes-or-less', 'time-to-make', 'course...","[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]",3,['pick whichever topping you want to use and c...,these toppings sure makes a nice change from p...,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...",13


# Change date format (from str to date format)

In [6]:
from datetime import datetime

dfinal['date'] = dfinal['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date())

In [7]:
print(type(dfinal['date'].iloc[0]))

<class 'datetime.date'>


# Categorising of recipes
Using tags to extract categories

In [8]:
def str_to_list(row):

    x = row[1:-1] # remove []
    x = x.split(',')
    x = [x.strip()[1:-1] for x in x]
        
    return x

In [9]:
df_tags = dfinal[['recipe_id','tags']]
df_tags.drop_duplicates(inplace=True)
df_tags.reset_index(drop=True,inplace=True) # reset index
df_tags.head()

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,recipe_id,tags
0,40893,"['weeknight', 'time-to-make', 'course', 'main-..."
1,44394,"['30-minutes-or-less', 'time-to-make', 'course..."
2,85009,"['15-minutes-or-less', 'time-to-make', 'course..."
3,120345,"['15-minutes-or-less', 'time-to-make', 'course..."
4,134728,"['60-minutes-or-less', 'time-to-make', 'main-i..."


In [10]:
df_tags['tag_list'] = df_tags['tags'].apply(lambda x: str_to_list(x))

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
df_tags.head()

,recipe_id,tags,tag_list
0,40893,"['weeknight', 'time-to-make', 'course', 'main-...","[weeknight, time-to-make, course, main-ingredi..."
1,44394,"['30-minutes-or-less', 'time-to-make', 'course...","[30-minutes-or-less, time-to-make, course, mai..."
2,85009,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai..."
3,120345,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai..."
4,134728,"['60-minutes-or-less', 'time-to-make', 'main-i...","[60-minutes-or-less, time-to-make, main-ingred..."


### Find unique tags

In [12]:
all_tags = list()
for tags in df_tags['tag_list']:
    for t in tags:
        if t not in all_tags:
            print(t)
            all_tags.append(t)

weeknight
time-to-make
course
main-ingredient
preparation
occasion
soups-stews
beans
vegetables
easy
crock-pot-slow-cooker
dietary
equipment
30-minutes-or-less
for-large-groups
5-ingredients-or-less
drop-cookies
desserts
lunch
oven
potluck
fall
finger-food
heirloom-historical
holiday-event
kid-friendly
picnic
spring
summer
winter
cakes
cookies-and-brownies
easter
chocolate
gifts
christmas
hanukkah
new-years
seasonal
comfort-food
independence-day
ramadan
rosh-hashana
valentines-day
taste-mood
to-go
number-of-servings
presentation
15-minutes-or-less
condiments-etc
inexpensive
3-steps-or-less
cuisine
north-american
low-protein
healthy
jams-and-preserves
fruit
american
low-fat
low-sodium
low-cholesterol
low-saturated-fat
healthy-2
northeastern-united-states
low-in-something
berries
raspberries
60-minutes-or-less
poultry
chicken
high-protein
high-in-something
meat
main-dish
beef
mexican
low-carb
lamb-sheep
african
moroccan
stove-top
spicy
4-hours-or-less
for-1-or-2
casseroles
southwestern-u

In [13]:
# Selected category
category = ['meat', 'seafood', 'salads', 'vegetables', 'soup', 'pasta', 'desserts', 'breads', 'burgers', 'beverages']

In [14]:
# Creating new column
df_tags['category'] = ""
df_tags

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,recipe_id,tags,tag_list,category
0,40893,"['weeknight', 'time-to-make', 'course', 'main-...","[weeknight, time-to-make, course, main-ingredi...",
1,44394,"['30-minutes-or-less', 'time-to-make', 'course...","[30-minutes-or-less, time-to-make, course, mai...",
2,85009,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",
3,120345,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",
4,134728,"['60-minutes-or-less', 'time-to-make', 'main-i...","[60-minutes-or-less, time-to-make, main-ingred...",
...,...,...,...,...
231632,470995,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",
231633,241491,"['course', 'main-ingredient', 'cuisine', 'prep...","[course, main-ingredient, cuisine, preparation...",
231634,257796,"['main-ingredient', 'preparation', 'occasion',...","[main-ingredient, preparation, occasion, poult...",
231635,72730,"['30-minutes-or-less', 'time-to-make', 'course...","[30-minutes-or-less, time-to-make, course, mai...",


In [15]:
for index, row in df_tags.iterrows():
    
    # if the recipe falls under multiple categories, use the first category found in the tag
    if df_tags.at[index, 'category'] == "": 
        
        for cat in category: 
            if cat in row['tags']:
                df_tags.at[index, 'category'] = cat    

In [16]:
df_tags

,recipe_id,tags,tag_list,category
0,40893,"['weeknight', 'time-to-make', 'course', 'main-...","[weeknight, time-to-make, course, main-ingredi...",soup
1,44394,"['30-minutes-or-less', 'time-to-make', 'course...","[30-minutes-or-less, time-to-make, course, mai...",desserts
2,85009,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",vegetables
3,120345,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",desserts
4,134728,"['60-minutes-or-less', 'time-to-make', 'main-i...","[60-minutes-or-less, time-to-make, main-ingred...",meat
...,...,...,...,...
231632,470995,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",seafood
231633,241491,"['course', 'main-ingredient', 'cuisine', 'prep...","[course, main-ingredient, cuisine, preparation...",
231634,257796,"['main-ingredient', 'preparation', 'occasion',...","[main-ingredient, preparation, occasion, poult...",meat
231635,72730,"['30-minutes-or-less', 'time-to-make', 'course...","[30-minutes-or-less, time-to-make, course, mai...",


In [17]:
df_tags['category'].value_counts()

desserts      42204
vegetables    40108
              37321
meat          36910
pasta         23523
breads        14536
beverages     11124
soup          10081
seafood        9679
salads         5872
burgers         279
Name: category, dtype: int64

### Drop recipes that do not fall in any category

In [18]:
df_tags = df_tags[df_tags['category'] != ""]
df_tags

,recipe_id,tags,tag_list,category
0,40893,"['weeknight', 'time-to-make', 'course', 'main-...","[weeknight, time-to-make, course, main-ingredi...",soup
1,44394,"['30-minutes-or-less', 'time-to-make', 'course...","[30-minutes-or-less, time-to-make, course, mai...",desserts
2,85009,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",vegetables
3,120345,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",desserts
4,134728,"['60-minutes-or-less', 'time-to-make', 'main-i...","[60-minutes-or-less, time-to-make, main-ingred...",meat
...,...,...,...,...
231629,492861,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, cui...",beverages
231630,273409,"['60-minutes-or-less', 'time-to-make', 'course...","[60-minutes-or-less, time-to-make, course, mai...",vegetables
231631,249924,"['time-to-make', 'course', 'main-ingredient', ...","[time-to-make, course, main-ingredient, cuisin...",meat
231632,470995,"['15-minutes-or-less', 'time-to-make', 'course...","[15-minutes-or-less, time-to-make, course, mai...",seafood


In [19]:
df_tags['category'].value_counts()

desserts      42204
vegetables    40108
meat          36910
pasta         23523
breads        14536
beverages     11124
soup          10081
seafood        9679
salads         5872
burgers         279
Name: category, dtype: int64

In [20]:
df_tags.drop(columns = ['tags', 'tag_list'], inplace=True)

C:\Users\Gi Han\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [21]:
df_tags

,recipe_id,category
0,40893,soup
1,44394,desserts
2,85009,vegetables
3,120345,desserts
4,134728,meat
...,...,...
231629,492861,beverages
231630,273409,vegetables
231631,249924,meat
231632,470995,seafood


# Extract nutritional Components

- from string to respective components
- convert them to float type
- normalise using log

In [22]:
df_nutri = dfinal[['recipe_id', 'name','n_ingredients','minutes','nutrition']]
df_nutri.drop_duplicates(inplace=True)
df_nutri.reset_index(drop=True,inplace=True)
df_nutri

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,recipe_id,name,n_ingredients,minutes,nutrition
0,40893,white bean green chile pepper soup,9,495,"[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]"
1,44394,devilicious cookie cake delights,4,20,"[132.3, 11.0, 39.0, 5.0, 4.0, 11.0, 5.0]"
2,85009,baked potato toppings,13,10,"[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]"
3,120345,sugared raspberries,2,10,"[838.0, 1.0, 820.0, 0.0, 2.0, 0.0, 71.0]"
4,134728,kfc honey bbq strips,12,40,"[316.0, 4.0, 40.0, 37.0, 78.0, 4.0, 10.0]"
...,...,...,...,...,...
231632,470995,perfectly fried shrimps with garlic,3,8,"[397.0, 62.0, 0.0, 8.0, 10.0, 28.0, 1.0]"
231633,241491,maple apple baked beans,10,375,"[518.1, 29.0, 138.0, 26.0, 34.0, 35.0, 24.0]"
231634,257796,slow cooker garlic chicken with rosemary,9,440,"[566.2, 59.0, 12.0, 15.0, 86.0, 55.0, 3.0]"
231635,72730,cranberry peach maple relish,6,22,"[1941.7, 3.0, 1569.0, 3.0, 10.0, 1.0, 154.0]"


In [23]:
df_nutri[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']] = df_nutri.nutrition.str.split(",",expand=True)

C:\Users\Gi Han\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [24]:
df_nutri['calories'] =  df_nutri['calories'].apply(lambda x: x.replace('[','')) 
df_nutri['carbohydrates (PDV)'] =  df_nutri['carbohydrates (PDV)'].apply(lambda x: x.replace(']',''))

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
df_nutri[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']] = df_nutri[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']].astype('float')

In [26]:
print(df_nutri.shape)
df_nutri.drop(['nutrition'], axis=1,inplace=True)
df_nutri.head()

(231637, 12)


,recipe_id,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
0,40893,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,2.0,10.0
1,44394,devilicious cookie cake delights,4,20,132.3,11.0,39.0,5.0,4.0,11.0,5.0
2,85009,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,301.0,42.0
3,120345,sugared raspberries,2,10,838.0,1.0,820.0,0.0,2.0,0.0,71.0
4,134728,kfc honey bbq strips,12,40,316.0,4.0,40.0,37.0,78.0,4.0,10.0


### Normalising the data using log

In [27]:
df_nutri.columns

Index(['recipe_id', 'name', 'n_ingredients', 'minutes', 'calories',
       'total fat (PDV)', 'sugar (PDV)', 'sodium (PDV)', 'protein (PDV)',
       'saturated fat (PDV)', 'carbohydrates (PDV)'],
      dtype='object')

In [28]:
lg_nutri = df_nutri[['recipe_id', 'calories', 'total fat (PDV)', 'sugar (PDV)', 'sodium (PDV)', 'protein (PDV)',
                        'saturated fat (PDV)', 'carbohydrates (PDV)']]

nutri = ['calories', 'total fat (PDV)', 'sugar (PDV)', 'sodium (PDV)', 'protein (PDV)', 
         'saturated fat (PDV)', 'carbohydrates (PDV)']

for n in nutri:
    lg_nutri[n] = lg_nutri[n].apply(lambda x: np.log(x))
    
lg_nutri

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,recipe_id,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
0,40893,5.322034,1.609438,2.197225,3.258097,3.178054,0.693147,2.302585
1,44394,4.885072,2.397895,3.663562,1.609438,1.386294,2.397895,1.609438
2,85009,7.932434,5.834811,4.897840,5.669881,5.081404,5.707110,3.737670
3,120345,6.731018,0.000000,6.709304,-inf,0.693147,-inf,4.262680
4,134728,5.755742,1.386294,3.688879,3.610918,4.356709,1.386294,2.302585
...,...,...,...,...,...,...,...,...
231632,470995,5.983936,4.127134,-inf,2.079442,2.302585,3.332205,0.000000
231633,241491,6.250168,3.367296,4.927254,3.258097,3.526361,3.555348,3.178054
231634,257796,6.338947,4.077537,2.484907,2.708050,4.454347,4.007333,1.098612
231635,72730,7.571319,1.098612,7.358194,1.098612,2.302585,0.000000,5.036953


In [29]:
# replace -inf with 0 
lg_nutri.replace(-np.inf, float(0), inplace=True)
lg_nutri

C:\Users\Gi Han\Anaconda3\lib\site-packages\pandas\core\frame.py:4263: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,recipe_id,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
0,40893,5.322034,1.609438,2.197225,3.258097,3.178054,0.693147,2.302585
1,44394,4.885072,2.397895,3.663562,1.609438,1.386294,2.397895,1.609438
2,85009,7.932434,5.834811,4.897840,5.669881,5.081404,5.707110,3.737670
3,120345,6.731018,0.000000,6.709304,0.000000,0.693147,0.000000,4.262680
4,134728,5.755742,1.386294,3.688879,3.610918,4.356709,1.386294,2.302585
...,...,...,...,...,...,...,...,...
231632,470995,5.983936,4.127134,0.000000,2.079442,2.302585,3.332205,0.000000
231633,241491,6.250168,3.367296,4.927254,3.258097,3.526361,3.555348,3.178054
231634,257796,6.338947,4.077537,2.484907,2.708050,4.454347,4.007333,1.098612
231635,72730,7.571319,1.098612,7.358194,1.098612,2.302585,0.000000,5.036953


In [30]:
lg_nutri.columns

Index(['recipe_id', 'calories', 'total fat (PDV)', 'sugar (PDV)',
       'sodium (PDV)', 'protein (PDV)', 'saturated fat (PDV)',
       'carbohydrates (PDV)'],
      dtype='object')

In [31]:
# rename lg_nutri columns
lg_nutri.rename(columns ={'calories': 'log calories', 'total fat (PDV)': 'log fat', 'sugar (PDV)': 'log sugar', 
                 'sodium (PDV)': 'log sodium', 'protein (PDV)': 'log protein', 'saturated fat (PDV)': 'log sat fat',
                 'carbohydrates (PDV)': 'log carbs'}, inplace=True)

lg_nutri

C:\Users\Gi Han\Anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,recipe_id,log calories,log fat,log sugar,log sodium,log protein,log sat fat,log carbs
0,40893,5.322034,1.609438,2.197225,3.258097,3.178054,0.693147,2.302585
1,44394,4.885072,2.397895,3.663562,1.609438,1.386294,2.397895,1.609438
2,85009,7.932434,5.834811,4.897840,5.669881,5.081404,5.707110,3.737670
3,120345,6.731018,0.000000,6.709304,0.000000,0.693147,0.000000,4.262680
4,134728,5.755742,1.386294,3.688879,3.610918,4.356709,1.386294,2.302585
...,...,...,...,...,...,...,...,...
231632,470995,5.983936,4.127134,0.000000,2.079442,2.302585,3.332205,0.000000
231633,241491,6.250168,3.367296,4.927254,3.258097,3.526361,3.555348,3.178054
231634,257796,6.338947,4.077537,2.484907,2.708050,4.454347,4.007333,1.098612
231635,72730,7.571319,1.098612,7.358194,1.098612,2.302585,0.000000,5.036953


In [32]:
# Combine with recipe data
df_nutri = df_nutri.merge(lg_nutri, on='recipe_id')
df_nutri

,recipe_id,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),log calories,log fat,log sugar,log sodium,log protein,log sat fat,log carbs
0,40893,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,2.0,10.0,5.322034,1.609438,2.197225,3.258097,3.178054,0.693147,2.302585
1,44394,devilicious cookie cake delights,4,20,132.3,11.0,39.0,5.0,4.0,11.0,5.0,4.885072,2.397895,3.663562,1.609438,1.386294,2.397895,1.609438
2,85009,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,301.0,42.0,7.932434,5.834811,4.897840,5.669881,5.081404,5.707110,3.737670
3,120345,sugared raspberries,2,10,838.0,1.0,820.0,0.0,2.0,0.0,71.0,6.731018,0.000000,6.709304,0.000000,0.693147,0.000000,4.262680
4,134728,kfc honey bbq strips,12,40,316.0,4.0,40.0,37.0,78.0,4.0,10.0,5.755742,1.386294,3.688879,3.610918,4.356709,1.386294,2.302585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231632,470995,perfectly fried shrimps with garlic,3,8,397.0,62.0,0.0,8.0,10.0,28.0,1.0,5.983936,4.127134,0.000000,2.079442,2.302585,3.332205,0.000000
231633,241491,maple apple baked beans,10,375,518.1,29.0,138.0,26.0,34.0,35.0,24.0,6.250168,3.367296,4.927254,3.258097,3.526361,3.555348,3.178054
231634,257796,slow cooker garlic chicken with rosemary,9,440,566.2,59.0,12.0,15.0,86.0,55.0,3.0,6.338947,4.077537,2.484907,2.708050,4.454347,4.007333,1.098612
231635,72730,cranberry peach maple relish,6,22,1941.7,3.0,1569.0,3.0,10.0,1.0,154.0,7.571319,1.098612,7.358194,1.098612,2.302585,0.000000,5.036953


# Combine dataset 
Merge with user information

In [33]:
dfuser = dfinal[['user_id', 'date', 'review','rating','recipe_id', 'description']]
dfuser.drop_duplicates(inplace=True)
dfuser.reset_index(drop=True,inplace=True)
dfuser

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,user_id,date,review,rating,recipe_id,description
0,38094,2003-02-17,Great with a salad. Cooked on top of stove for...,4,40893,easy soup for the crockpot.
1,1293707,2011-12-21,"So simple, so delicious! Great for chilly fall...",5,40893,easy soup for the crockpot.
2,8937,2002-12-01,This worked very well and is EASY. I used not...,4,44394,NaN
3,126440,2010-02-27,I made the Mexican topping and took it to bunk...,5,85009,these toppings sure makes a nice change from p...
4,57222,2011-10-01,"Made the cheddar bacon topping, adding a sprin...",5,85009,these toppings sure makes a nice change from p...
...,...,...,...,...,...,...
1132362,417131,2007-09-05,I made this over the weekend and changed it up...,4,241491,i've been looking for a perfect baked bean rec...
1132363,648540,2010-09-06,"Fabulous! We are vegetarian, so I used vegetar...",5,241491,i've been looking for a perfect baked bean rec...
1132364,1269180,2011-08-07,Very tender and juicy - had trouble getting t...,4,257796,delicious and easy!
1132365,116593,2003-12-09,Another approach is to start making sauce with...,0,72730,great simple alternative to regular cranberry ...


In [34]:
df_combine = df_nutri.merge(dfuser, how='inner', on='recipe_id')
df_combine

,recipe_id,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
0,40893,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,2.0,...,2.197225,3.258097,3.178054,0.693147,2.302585,38094,2003-02-17,Great with a salad. Cooked on top of stove for...,4,easy soup for the crockpot.
1,40893,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,2.0,...,2.197225,3.258097,3.178054,0.693147,2.302585,1293707,2011-12-21,"So simple, so delicious! Great for chilly fall...",5,easy soup for the crockpot.
2,44394,devilicious cookie cake delights,4,20,132.3,11.0,39.0,5.0,4.0,11.0,...,3.663562,1.609438,1.386294,2.397895,1.609438,8937,2002-12-01,This worked very well and is EASY. I used not...,4,NaN
3,85009,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,301.0,...,4.897840,5.669881,5.081404,5.707110,3.737670,126440,2010-02-27,I made the Mexican topping and took it to bunk...,5,these toppings sure makes a nice change from p...
4,85009,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,301.0,...,4.897840,5.669881,5.081404,5.707110,3.737670,57222,2011-10-01,"Made the cheddar bacon topping, adding a sprin...",5,these toppings sure makes a nice change from p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132362,241491,maple apple baked beans,10,375,518.1,29.0,138.0,26.0,34.0,35.0,...,4.927254,3.258097,3.526361,3.555348,3.178054,417131,2007-09-05,I made this over the weekend and changed it up...,4,i've been looking for a perfect baked bean rec...
1132363,241491,maple apple baked beans,10,375,518.1,29.0,138.0,26.0,34.0,35.0,...,4.927254,3.258097,3.526361,3.555348,3.178054,648540,2010-09-06,"Fabulous! We are vegetarian, so I used vegetar...",5,i've been looking for a perfect baked bean rec...
1132364,257796,slow cooker garlic chicken with rosemary,9,440,566.2,59.0,12.0,15.0,86.0,55.0,...,2.484907,2.708050,4.454347,4.007333,1.098612,1269180,2011-08-07,Very tender and juicy - had trouble getting t...,4,delicious and easy!
1132365,72730,cranberry peach maple relish,6,22,1941.7,3.0,1569.0,3.0,10.0,1.0,...,7.358194,1.098612,2.302585,0.000000,5.036953,116593,2003-12-09,Another approach is to start making sauce with...,0,great simple alternative to regular cranberry ...


In [35]:
df_combine = df_tags.merge(df_combine, how='inner', on='recipe_id')
df_combine

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
0,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,...,2.197225,3.258097,3.178054,0.693147,2.302585,38094,2003-02-17,Great with a salad. Cooked on top of stove for...,4,easy soup for the crockpot.
1,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,...,2.197225,3.258097,3.178054,0.693147,2.302585,1293707,2011-12-21,"So simple, so delicious! Great for chilly fall...",5,easy soup for the crockpot.
2,44394,desserts,devilicious cookie cake delights,4,20,132.3,11.0,39.0,5.0,4.0,...,3.663562,1.609438,1.386294,2.397895,1.609438,8937,2002-12-01,This worked very well and is EASY. I used not...,4,NaN
3,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,...,4.897840,5.669881,5.081404,5.707110,3.737670,126440,2010-02-27,I made the Mexican topping and took it to bunk...,5,these toppings sure makes a nice change from p...
4,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,...,4.897840,5.669881,5.081404,5.707110,3.737670,57222,2011-10-01,"Made the cheddar bacon topping, adding a sprin...",5,these toppings sure makes a nice change from p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978894,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,2.944439,2.890372,4.143135,3.663562,2.079442,226863,2009-06-19,"I made this without the bread, and added some ...",5,"sweet and sticky with a kick of heat, this eas..."
978895,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,2.944439,2.890372,4.143135,3.663562,2.079442,39835,2010-05-15,This chicken was great (skipped the bread). My...,5,"sweet and sticky with a kick of heat, this eas..."
978896,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,2.944439,2.890372,4.143135,3.663562,2.079442,496803,2011-08-21,I really loved this! I cut the recipe back to...,5,"sweet and sticky with a kick of heat, this eas..."
978897,470995,seafood,perfectly fried shrimps with garlic,3,8,397.0,62.0,0.0,8.0,10.0,...,0.000000,2.079442,2.302585,3.332205,0.000000,169430,2013-04-17,Just as the title promised this is a lovely sh...,5,succulent shrimps fried in olive oil with cara...


In [36]:
df_combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 978899 entries, 0 to 978898
Data columns (total 24 columns):
recipe_id              978899 non-null int64
category               978899 non-null object
name                   978898 non-null object
n_ingredients          978899 non-null int64
minutes                978899 non-null int64
calories               978899 non-null float64
total fat (PDV)        978899 non-null float64
sugar (PDV)            978899 non-null float64
sodium (PDV)           978899 non-null float64
protein (PDV)          978899 non-null float64
saturated fat (PDV)    978899 non-null float64
carbohydrates (PDV)    978899 non-null float64
log calories           978899 non-null float64
log fat                978899 non-null float64
log sugar              978899 non-null float64
log sodium             978899 non-null float64
log protein            978899 non-null float64
log sat fat            978899 non-null float64
log carbs              978899 non-null float64
user

# Drop rows:

* without review
* without description
* description length less than 3
* without name

In [37]:
# drop rows without review, description, name

df_combine = df_combine.dropna(subset=['review', 'description', 'name'])
df_combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 957630 entries, 0 to 978898
Data columns (total 24 columns):
recipe_id              957630 non-null int64
category               957630 non-null object
name                   957630 non-null object
n_ingredients          957630 non-null int64
minutes                957630 non-null int64
calories               957630 non-null float64
total fat (PDV)        957630 non-null float64
sugar (PDV)            957630 non-null float64
sodium (PDV)           957630 non-null float64
protein (PDV)          957630 non-null float64
saturated fat (PDV)    957630 non-null float64
carbohydrates (PDV)    957630 non-null float64
log calories           957630 non-null float64
log fat                957630 non-null float64
log sugar              957630 non-null float64
log sodium             957630 non-null float64
log protein            957630 non-null float64
log sat fat            957630 non-null float64
log carbs              957630 non-null float64
user

In [38]:
# drop rows with descripion length less than 3

df_combine['description_token'] = df_combine['description'].apply(lambda x: x.strip().split())
df_combine.head()

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description,description_token
0,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,...,3.258097,3.178054,0.693147,2.302585,38094,2003-02-17,Great with a salad. Cooked on top of stove for...,4,easy soup for the crockpot.,"[easy, soup, for, the, crockpot.]"
1,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,...,3.258097,3.178054,0.693147,2.302585,1293707,2011-12-21,"So simple, so delicious! Great for chilly fall...",5,easy soup for the crockpot.,"[easy, soup, for, the, crockpot.]"
3,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,...,5.669881,5.081404,5.707110,3.737670,126440,2010-02-27,I made the Mexican topping and took it to bunk...,5,these toppings sure makes a nice change from p...,"[these, toppings, sure, makes, a, nice, change..."
4,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,...,5.669881,5.081404,5.707110,3.737670,57222,2011-10-01,"Made the cheddar bacon topping, adding a sprin...",5,these toppings sure makes a nice change from p...,"[these, toppings, sure, makes, a, nice, change..."
5,120345,desserts,sugared raspberries,2,10,838.0,1.0,820.0,0.0,2.0,...,0.000000,0.693147,0.000000,4.262680,52282,2005-05-21,very very sweet. after i waited the 2 days i b...,4,here's an old method for preserving fruit with...,"[here's, an, old, method, for, preserving, fru..."


In [39]:
df_combine['description_len'] = df_combine['description_token'].str.len()
df_combine

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log protein,log sat fat,log carbs,user_id,date,review,rating,description,description_token,description_len
0,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,...,3.178054,0.693147,2.302585,38094,2003-02-17,Great with a salad. Cooked on top of stove for...,4,easy soup for the crockpot.,"[easy, soup, for, the, crockpot.]",5
1,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,...,3.178054,0.693147,2.302585,1293707,2011-12-21,"So simple, so delicious! Great for chilly fall...",5,easy soup for the crockpot.,"[easy, soup, for, the, crockpot.]",5
3,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,...,5.081404,5.707110,3.737670,126440,2010-02-27,I made the Mexican topping and took it to bunk...,5,these toppings sure makes a nice change from p...,"[these, toppings, sure, makes, a, nice, change...",25
4,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,...,5.081404,5.707110,3.737670,57222,2011-10-01,"Made the cheddar bacon topping, adding a sprin...",5,these toppings sure makes a nice change from p...,"[these, toppings, sure, makes, a, nice, change...",25
5,120345,desserts,sugared raspberries,2,10,838.0,1.0,820.0,0.0,2.0,...,0.693147,0.000000,4.262680,52282,2005-05-21,very very sweet. after i waited the 2 days i b...,4,here's an old method for preserving fruit with...,"[here's, an, old, method, for, preserving, fru...",98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978894,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,4.143135,3.663562,2.079442,226863,2009-06-19,"I made this without the bread, and added some ...",5,"sweet and sticky with a kick of heat, this eas...","[sweet, and, sticky, with, a, kick, of, heat,,...",63
978895,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,4.143135,3.663562,2.079442,39835,2010-05-15,This chicken was great (skipped the bread). My...,5,"sweet and sticky with a kick of heat, this eas...","[sweet, and, sticky, with, a, kick, of, heat,,...",63
978896,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,4.143135,3.663562,2.079442,496803,2011-08-21,I really loved this! I cut the recipe back to...,5,"sweet and sticky with a kick of heat, this eas...","[sweet, and, sticky, with, a, kick, of, heat,,...",63
978897,470995,seafood,perfectly fried shrimps with garlic,3,8,397.0,62.0,0.0,8.0,10.0,...,2.302585,3.332205,0.000000,169430,2013-04-17,Just as the title promised this is a lovely sh...,5,succulent shrimps fried in olive oil with cara...,"[succulent, shrimps, fried, in, olive, oil, wi...",41


In [40]:
df_combine = df_combine[df_combine['description_len'] >= 3]
df_combine

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log protein,log sat fat,log carbs,user_id,date,review,rating,description,description_token,description_len
0,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,...,3.178054,0.693147,2.302585,38094,2003-02-17,Great with a salad. Cooked on top of stove for...,4,easy soup for the crockpot.,"[easy, soup, for, the, crockpot.]",5
1,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,...,3.178054,0.693147,2.302585,1293707,2011-12-21,"So simple, so delicious! Great for chilly fall...",5,easy soup for the crockpot.,"[easy, soup, for, the, crockpot.]",5
3,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,...,5.081404,5.707110,3.737670,126440,2010-02-27,I made the Mexican topping and took it to bunk...,5,these toppings sure makes a nice change from p...,"[these, toppings, sure, makes, a, nice, change...",25
4,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,...,5.081404,5.707110,3.737670,57222,2011-10-01,"Made the cheddar bacon topping, adding a sprin...",5,these toppings sure makes a nice change from p...,"[these, toppings, sure, makes, a, nice, change...",25
5,120345,desserts,sugared raspberries,2,10,838.0,1.0,820.0,0.0,2.0,...,0.693147,0.000000,4.262680,52282,2005-05-21,very very sweet. after i waited the 2 days i b...,4,here's an old method for preserving fruit with...,"[here's, an, old, method, for, preserving, fru...",98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978894,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,4.143135,3.663562,2.079442,226863,2009-06-19,"I made this without the bread, and added some ...",5,"sweet and sticky with a kick of heat, this eas...","[sweet, and, sticky, with, a, kick, of, heat,,...",63
978895,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,4.143135,3.663562,2.079442,39835,2010-05-15,This chicken was great (skipped the bread). My...,5,"sweet and sticky with a kick of heat, this eas...","[sweet, and, sticky, with, a, kick, of, heat,,...",63
978896,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,4.143135,3.663562,2.079442,496803,2011-08-21,I really loved this! I cut the recipe back to...,5,"sweet and sticky with a kick of heat, this eas...","[sweet, and, sticky, with, a, kick, of, heat,,...",63
978897,470995,seafood,perfectly fried shrimps with garlic,3,8,397.0,62.0,0.0,8.0,10.0,...,2.302585,3.332205,0.000000,169430,2013-04-17,Just as the title promised this is a lovely sh...,5,succulent shrimps fried in olive oil with cara...,"[succulent, shrimps, fried, in, olive, oil, wi...",41


In [41]:
df_combine = df_combine.drop(['description_token', 'description_len'], axis=1)
df_combine

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
0,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,...,2.197225,3.258097,3.178054,0.693147,2.302585,38094,2003-02-17,Great with a salad. Cooked on top of stove for...,4,easy soup for the crockpot.
1,40893,soup,white bean green chile pepper soup,9,495,204.8,5.0,9.0,26.0,24.0,...,2.197225,3.258097,3.178054,0.693147,2.302585,1293707,2011-12-21,"So simple, so delicious! Great for chilly fall...",5,easy soup for the crockpot.
3,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,...,4.897840,5.669881,5.081404,5.707110,3.737670,126440,2010-02-27,I made the Mexican topping and took it to bunk...,5,these toppings sure makes a nice change from p...
4,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,...,4.897840,5.669881,5.081404,5.707110,3.737670,57222,2011-10-01,"Made the cheddar bacon topping, adding a sprin...",5,these toppings sure makes a nice change from p...
5,120345,desserts,sugared raspberries,2,10,838.0,1.0,820.0,0.0,2.0,...,6.709304,0.000000,0.693147,0.000000,4.262680,52282,2005-05-21,very very sweet. after i waited the 2 days i b...,4,here's an old method for preserving fruit with...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978894,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,2.944439,2.890372,4.143135,3.663562,2.079442,226863,2009-06-19,"I made this without the bread, and added some ...",5,"sweet and sticky with a kick of heat, this eas..."
978895,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,2.944439,2.890372,4.143135,3.663562,2.079442,39835,2010-05-15,This chicken was great (skipped the bread). My...,5,"sweet and sticky with a kick of heat, this eas..."
978896,249924,meat,rocky mount nc bbq chicken sandwiches,5,70,505.9,44.0,19.0,18.0,63.0,...,2.944439,2.890372,4.143135,3.663562,2.079442,496803,2011-08-21,I really loved this! I cut the recipe back to...,5,"sweet and sticky with a kick of heat, this eas..."
978897,470995,seafood,perfectly fried shrimps with garlic,3,8,397.0,62.0,0.0,8.0,10.0,...,0.000000,2.079442,2.302585,3.332205,0.000000,169430,2013-04-17,Just as the title promised this is a lovely sh...,5,succulent shrimps fried in olive oil with cara...


### Filtering data
- User that has reviewed 250 <= reviews <= 1000 reviews

In [42]:
user_filter = df_combine.groupby("user_id").filter(lambda x: 250 <= len(x)  <= 1000)

In [43]:
user_filter

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
4,85009,vegetables,baked potato toppings,13,10,2786.2,342.0,134.0,290.0,161.0,...,4.897840,5.669881,5.081404,5.707110,3.737670,57222,2011-10-01,"Made the cheddar bacon topping, adding a sprin...",5,these toppings sure makes a nice change from p...
5,120345,desserts,sugared raspberries,2,10,838.0,1.0,820.0,0.0,2.0,...,6.709304,0.000000,0.693147,0.000000,4.262680,52282,2005-05-21,very very sweet. after i waited the 2 days i b...,4,here's an old method for preserving fruit with...
6,120345,desserts,sugared raspberries,2,10,838.0,1.0,820.0,0.0,2.0,...,6.709304,0.000000,0.693147,0.000000,4.262680,124416,2011-08-06,"Just an observation, so I will not rate. I fo...",0,here's an old method for preserving fruit with...
8,134728,meat,kfc honey bbq strips,12,40,316.0,4.0,40.0,37.0,78.0,...,3.688879,3.610918,4.356709,1.386294,2.302585,76535,2005-09-02,Very good!,4,these are so yummy and they do taste just like...
13,134728,meat,kfc honey bbq strips,12,40,316.0,4.0,40.0,37.0,78.0,...,3.688879,3.610918,4.356709,1.386294,2.302585,255338,2008-04-11,First time using liquid smoke in a recipe. Mad...,5,these are so yummy and they do taste just like...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978871,40514,vegetables,sun dried tomato bruschetta with goat cheese,5,15,94.6,5.0,4.0,8.0,8.0,...,1.386294,2.079442,2.079442,2.302585,1.098612,469903,2008-06-24,Simple flavors of summertime! Instead of bread...,4,these are good. if you haven't tried goat chee...
978873,40514,vegetables,sun dried tomato bruschetta with goat cheese,5,15,94.6,5.0,4.0,8.0,8.0,...,1.386294,2.079442,2.079442,2.302585,1.098612,2000431901,2017-11-25,SO good! I ate 4 of them fresh from the oven!,5,these are good. if you haven't tried goat chee...
978882,190698,salads,strawberry balsamic vinaigrette,4,5,102.1,15.0,8.0,0.0,0.0,...,2.079442,0.000000,0.000000,1.791759,0.000000,145352,2007-05-14,Oh I loved this! So quick and so tasty! I made...,5,saw this on rachel rey 30 min meals! she said ...
978886,190698,salads,strawberry balsamic vinaigrette,4,5,102.1,15.0,8.0,0.0,0.0,...,2.079442,0.000000,0.000000,1.791759,0.000000,333017,2009-06-24,"This was just OK. When I first tasted it, I fo...",3,saw this on rachel rey 30 min meals! she said ...


In [44]:
user_filter['user_id'].value_counts()

61660     982
56003     975
145352    968
537937    954
171790    952
         ... 
177443    251
398275    251
255338    251
330505    250
512309    250
Name: user_id, Length: 386, dtype: int64

### Sort dataframe by date

In [45]:
final = user_filter.sort_values(by="date")
final.drop_duplicates(inplace=True)
print(final.shape)
final.head()

(173927, 24)


,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
839094,9014,vegetables,cheesy potato skillet stuff,5,40,357.8,33.0,6.0,22.0,30.0,...,1.791759,3.091042,3.401197,3.610918,2.079442,11297,2001-06-18,What a great quick & easy meal especially for ...,4,this recipe came about when a friend of mine r...
854757,9492,meat,chicken honey glazed,7,100,484.9,11.0,189.0,20.0,110.0,...,5.241747,2.995732,4.700480,1.791759,2.772589,11297,2001-06-26,So easy and tasty. To make the clean-up even...,4,serve with a green salad and a your favorite r...
526859,2731,meat,jaegerschnitzel,17,35,382.2,28.0,16.0,17.0,61.0,...,2.772589,2.833213,4.110874,3.526361,1.609438,11297,2001-07-02,The man of the house really loved the sauce wi...,4,(breaded veal cutlet with mushroom sauce)
25366,10104,pasta,broccoli pork,12,30,150.3,8.0,19.0,26.0,30.0,...,2.944439,3.258097,3.401197,1.791759,1.098612,11297,2001-07-16,Really great. Sometimes I also add some thinly...,4,"this is my favorite marinade for pork, orienta..."
747124,5478,meat,easy garlic chicken,5,30,221.3,9.0,53.0,5.0,50.0,...,3.970292,1.609438,3.912023,1.609438,1.386294,11297,2001-07-17,Easy and tasty. Even easier clean-up by lini...,4,"a quick, easy chicken recipe for days when you..."


In [46]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173927 entries, 839094 to 441516
Data columns (total 24 columns):
recipe_id              173927 non-null int64
category               173927 non-null object
name                   173927 non-null object
n_ingredients          173927 non-null int64
minutes                173927 non-null int64
calories               173927 non-null float64
total fat (PDV)        173927 non-null float64
sugar (PDV)            173927 non-null float64
sodium (PDV)           173927 non-null float64
protein (PDV)          173927 non-null float64
saturated fat (PDV)    173927 non-null float64
carbohydrates (PDV)    173927 non-null float64
log calories           173927 non-null float64
log fat                173927 non-null float64
log sugar              173927 non-null float64
log sodium             173927 non-null float64
log protein            173927 non-null float64
log sat fat            173927 non-null float64
log carbs              173927 non-null float64

In [47]:
# Convert to csv
final.to_csv('./Dataset/final_dataset.csv', index=False)

# Train & Test Dataset
75-25 train-test split

In [140]:
all_dataset = pd.read_csv('./Dataset/final_dataset.csv')
all_dataset.head()

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
0,9014,vegetables,cheesy potato skillet stuff,5,40,357.8,33.0,6.0,22.0,30.0,...,1.791759,3.091042,3.401197,3.610918,2.079442,11297,2001-06-18,What a great quick & easy meal especially for ...,4,this recipe came about when a friend of mine r...
1,9492,meat,chicken honey glazed,7,100,484.9,11.0,189.0,20.0,110.0,...,5.241747,2.995732,4.700480,1.791759,2.772589,11297,2001-06-26,So easy and tasty. To make the clean-up even...,4,serve with a green salad and a your favorite r...
2,2731,meat,jaegerschnitzel,17,35,382.2,28.0,16.0,17.0,61.0,...,2.772589,2.833213,4.110874,3.526361,1.609438,11297,2001-07-02,The man of the house really loved the sauce wi...,4,(breaded veal cutlet with mushroom sauce)
3,10104,pasta,broccoli pork,12,30,150.3,8.0,19.0,26.0,30.0,...,2.944439,3.258097,3.401197,1.791759,1.098612,11297,2001-07-16,Really great. Sometimes I also add some thinly...,4,"this is my favorite marinade for pork, orienta..."
4,5478,meat,easy garlic chicken,5,30,221.3,9.0,53.0,5.0,50.0,...,3.970292,1.609438,3.912023,1.609438,1.386294,11297,2001-07-17,Easy and tasty. Even easier clean-up by lini...,4,"a quick, easy chicken recipe for days when you..."


In [141]:
def train_test_split(df):

    import math

    date = df.iloc[math.floor(.70 * len(df))]['date']
    print('Date of the 75th-percentile review: ', date)  

    i_date = df.index[df['date'] == date].tolist()
    print(f'Index of the first review on {date}: ', i_date[0]) 

    train_set = df.iloc[:i_date[0]]
    print(train_set.shape)

    test_set = df.iloc[i_date[0]:]
    print(test_set.shape)

    return (train_set, test_set)


In [142]:
train_set, test_set = train_test_split(all_dataset)

Date of the 75th-percentile review:  2009-10-03
Index of the first review on 2009-10-03:  121668
(121668, 24)
(52259, 24)


In [143]:
train_set.tail()

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
121663,313101,salads,asian beef salad,7,20,312.1,22.0,14.0,7.0,70.0,...,2.639057,1.945910,4.248495,3.218876,1.098612,114027,2009-10-02,This was a really nice salad! I loved how refr...,5,"this is a delicious, low-calorie salad adapted..."
121664,258429,meat,corned beef corned silverside for the crock pot,5,490,652.8,73.0,20.0,118.0,91.0,...,2.995732,4.770685,4.510860,4.369448,0.693147,58439,2009-10-02,"Beautiful delicious, TERRIFICALLY moist, TRUE!...",5,beautiful slow cooked corned meat. put it on i...
121665,83475,meat,three cup chicken,10,60,668.4,60.0,43.0,62.0,104.0,...,3.761200,4.127134,4.644391,3.761200,1.791759,185926,2009-10-02,I love this chicken and I don't even like chic...,5,this is a traditional taiwanese dish that we a...
121666,388912,vegetables,roasted mushrooms baked with a bourbon cream s...,16,55,255.9,24.0,14.0,22.0,10.0,...,2.639057,3.091042,2.302585,3.526361,1.609438,690623,2009-10-02,We really loved these mushrooms! I would howe...,5,this is a favorite of mine. roasted rich cremi...
121667,55384,beverages,frozen grasshopper,5,5,625.4,44.0,259.0,8.0,18.0,...,5.556828,2.079442,2.890372,4.488636,3.135494,552613,2009-10-02,"Wow, this was just about perfect for my tasteb...",5,i found this in with my grandmothers recipes. ...


In [144]:
test_set.head()

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
121668,392210,meat,chicken paprikash hungarian style i think,10,80,527.5,65.0,10.0,21.0,49.0,...,2.302585,3.044522,3.891820,4.682131,1.386294,339260,2009-10-03,This was the first time I had ever had Chicken...,4,this is how my grandma and great grandma have ...
121669,347466,vegetables,spinach saute with garlic and parmesan cheese,7,10,23.3,2.0,0.0,1.0,2.0,...,0.000000,0.000000,0.693147,0.000000,0.000000,470351,2009-10-03,VERY very good! Followed this recipe as state...,4,here is another recipe from the www.lifescript...
121670,45348,pasta,basil walnut pesto,7,12,213.8,32.0,1.0,5.0,11.0,...,0.000000,1.609438,2.397895,2.890372,0.000000,706608,2009-10-03,Just like most I tried this recipe because I n...,4,this is my favorite pesto recipe; rich but hea...
121671,330059,pasta,bulgur and chickpeas with preserved lemon vina...,17,25,225.2,12.0,6.0,24.0,13.0,...,1.791759,3.178054,2.564949,1.609438,2.397895,621626,2009-10-03,DH really enjoyed this dish. I did leave out t...,5,this dish is loaded with texture and flavor. w...
121672,279553,soup,turkey and brown rice chilli,14,485,202.4,2.0,37.0,22.0,34.0,...,3.610918,3.091042,3.526361,0.000000,2.302585,621626,2009-10-03,I made this on the stove top since it was a la...,5,fantstic crock pot recipe fron betty crocker. ...


## Check all users in test set is available in train set
Remove new users in test set

In [145]:
userid_train = train_set['user_id'].unique()
userid_train[:10]

array([11297, 13483,  8629, 13796, 18391, 20480, 21752, 13593,  7108,
       22015], dtype=int64)

In [146]:
userid_test = test_set['user_id'].unique()
userid_test[:10]

array([ 339260,  470351,  706608,  621626,  724516,  599450,  705251,
         56003,  137302, 1366254], dtype=int64)

In [147]:
new_user = []
for i in userid_test:
    if i not in userid_train:
        new_user.append(i)
        
new_user

[1701315, 1680722, 1706426, 1925885, 2123645, 2324285, 1802849661, 2000431901]

In [148]:
# Drop new users in test dataset
test_set = test_set[~test_set['user_id'].isin(new_user)]
test_set

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
121668,392210,meat,chicken paprikash hungarian style i think,10,80,527.5,65.0,10.0,21.0,49.0,...,2.302585,3.044522,3.891820,4.682131,1.386294,339260,2009-10-03,This was the first time I had ever had Chicken...,4,this is how my grandma and great grandma have ...
121669,347466,vegetables,spinach saute with garlic and parmesan cheese,7,10,23.3,2.0,0.0,1.0,2.0,...,0.000000,0.000000,0.693147,0.000000,0.000000,470351,2009-10-03,VERY very good! Followed this recipe as state...,4,here is another recipe from the www.lifescript...
121670,45348,pasta,basil walnut pesto,7,12,213.8,32.0,1.0,5.0,11.0,...,0.000000,1.609438,2.397895,2.890372,0.000000,706608,2009-10-03,Just like most I tried this recipe because I n...,4,this is my favorite pesto recipe; rich but hea...
121671,330059,pasta,bulgur and chickpeas with preserved lemon vina...,17,25,225.2,12.0,6.0,24.0,13.0,...,1.791759,3.178054,2.564949,1.609438,2.397895,621626,2009-10-03,DH really enjoyed this dish. I did leave out t...,5,this dish is loaded with texture and flavor. w...
121672,279553,soup,turkey and brown rice chilli,14,485,202.4,2.0,37.0,22.0,34.0,...,3.610918,3.091042,3.526361,0.000000,2.302585,621626,2009-10-03,I made this on the stove top since it was a la...,5,fantstic crock pot recipe fron betty crocker. ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173921,23113,pasta,pumpkin risotto,12,30,405.6,16.0,8.0,9.0,22.0,...,2.079442,2.197225,3.091042,3.135494,3.044522,198154,2018-12-11,I made this as a side because DH is a meat and...,4,my ultimate comfort food. i make this so often...
173923,242547,meat,pulled chicken sandwiches crock pot,12,370,207.4,4.0,81.0,35.0,40.0,...,4.394449,3.555348,3.688879,0.693147,2.079442,218535,2018-12-16,amazing. one of the best crockpot bbq recipes,5,"a personal take on a recipe from the ""slow coo..."
173924,108068,pasta,velveeta macaroni and cheese,6,55,348.1,27.0,16.0,30.0,26.0,...,2.772589,3.401197,3.258097,3.761200,2.397895,218535,2018-12-16,I did not put the cheese slices on top. Still ...,5,"my son is a very picky eater, but loves velvee..."
173925,167834,vegetables,bacon wrapped asparagus,2,25,24.0,2.0,2.0,1.0,2.0,...,0.693147,0.000000,0.693147,1.098612,0.000000,218535,2018-12-16,so delicious that it's gone in the blink of an...,5,easy-to-prepare appetizer. laurita has a good...


## Check all users in test set has reviewed at least 10 recipes
Can compare Top 10 recipes in the testing stage of the recommender system

In [149]:
review_cnt = test_set['user_id'].value_counts()
review_cnt

266635    909
895132    780
573325    658
844554    637
865936    595
         ... 
2310        1
35140       1
25792       1
55380       1
52125       1
Name: user_id, Length: 361, dtype: int64

In [150]:
# check if each recipe is only reviewed once by each user
cnt_check = test_set.groupby(['user_id', 'recipe_id']).count()
cnt_check

category  name  n_ingredients  minutes  calories  \
user_id recipe_id                                                     
1535    1267              1     1              1        1         1   
        10125             1     1              1        1         1   
        10465             1     1              1        1         1   
        15859             1     1              1        1         1   
        16533             1     1              1        1         1   
...                     ...   ...            ...      ...       ...   
1366254 453628            1     1              1        1         1   
        453789            1     1              1        1         1   
        453970            1     1              1        1         1   
        456753            1     1              1        1         1   
        457687            1     1              1        1         1   

                   total fat (PDV)  sugar (PDV)  sodium (PDV)  protein (PDV)  \
user_id recipe_id                                                              
1535    1267                     1            1             1              1   
        10125                    1            1             1              1   
        10465                    1            1             1              1   
        15859                    1            1             1              1   
        16533                    1            1             1              1   
...                            ...          ...           ...            ...   
1366254 453628                   1            1             1              1   
        453789                   1            1             1              1   
        453970                   1            1             1              1   
        456753                   1            1             1              1   
        457687                   1            1             1              1   

                   saturated fat (PDV)  ...  log fat  log sugar  log sodium  \
user_id recipe_id                       ...                                   
1535    1267                         1  ...        1          1           1   
        10125                        1  ...        1          1           1   
        10465                        1  ...        1          1           1   
        15859                        1  ...        1          1           1   
        16533                        1  ...        1          1           1   
...                                ...  ...      ...        ...         ...   
1366254 453628                       1  ...        1          1           1   
        453789                       1  ...        1          1           1   
        453970                       1  ...        1          1           1   
        456753                       1  ...        1          1           1   
        457687                       1  ...        1          1           1   

                   log protein  log sat fat  log carbs  date  review  rating  \
user_id recipe_id                                                              
1535    1267                 1            1          1     1       1       1   
        10125                1            1          1     1       1       1   
        10465                1            1          1     1       1       1   
        15859                1            1          1     1       1       1   
        16533                1            1          1     1       1       1   
...                        ...          ...        ...   ...     ...     ...   
1366254 453628               1            1          1     1       1       1   
        453789               1            1          1     1       1       1   
        453970               1            1          1     1       1       1   
        456753               1            1          1     1       1       1   
        457687               1            1          1     1      

In [151]:
# since the number is equal to the number of in the test set, each user only reviewed the recipe once
cnt_check.sum() 

category               49111
name                   49111
n_ingredients          49111
minutes                49111
calories               49111
total fat (PDV)        49111
sugar (PDV)            49111
sodium (PDV)           49111
protein (PDV)          49111
saturated fat (PDV)    49111
carbohydrates (PDV)    49111
log calories           49111
log fat                49111
log sugar              49111
log sodium             49111
log protein            49111
log sat fat            49111
log carbs              49111
date                   49111
review                 49111
rating                 49111
description            49111
dtype: int64

In [152]:
less10 = []
for i in review_cnt.index:
    if review_cnt[i] < 10:
        less10.append(i)
        
print('no. of users (< 10 recipes): ', len(less10))
print(less10)

no. of users (< 10 recipes):  41
[58038, 52282, 113941, 125640, 32772, 512309, 402559, 18391, 283251, 141293, 134624, 52543, 35526, 209255, 27643, 147027, 21752, 27443, 288218, 49304, 86512, 124249, 171303, 126435, 269480, 351811, 193516, 36944, 248023, 73836, 209983, 65056, 185285, 206722, 129201, 52448, 2310, 35140, 25792, 55380, 52125]


In [153]:
# Drop user that reviewed < 10 recipes
test_set = test_set[~test_set['user_id'].isin(less10)]

In [154]:
test_set['user_id'].value_counts()

266635    909
895132    780
573325    658
844554    637
865936    595
         ... 
67103      11
37305      10
121690     10
237123     10
57256      10
Name: user_id, Length: 320, dtype: int64

In [155]:
test_set

,recipe_id,category,name,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),...,log sugar,log sodium,log protein,log sat fat,log carbs,user_id,date,review,rating,description
121668,392210,meat,chicken paprikash hungarian style i think,10,80,527.5,65.0,10.0,21.0,49.0,...,2.302585,3.044522,3.891820,4.682131,1.386294,339260,2009-10-03,This was the first time I had ever had Chicken...,4,this is how my grandma and great grandma have ...
121669,347466,vegetables,spinach saute with garlic and parmesan cheese,7,10,23.3,2.0,0.0,1.0,2.0,...,0.000000,0.000000,0.693147,0.000000,0.000000,470351,2009-10-03,VERY very good! Followed this recipe as state...,4,here is another recipe from the www.lifescript...
121670,45348,pasta,basil walnut pesto,7,12,213.8,32.0,1.0,5.0,11.0,...,0.000000,1.609438,2.397895,2.890372,0.000000,706608,2009-10-03,Just like most I tried this recipe because I n...,4,this is my favorite pesto recipe; rich but hea...
121671,330059,pasta,bulgur and chickpeas with preserved lemon vina...,17,25,225.2,12.0,6.0,24.0,13.0,...,1.791759,3.178054,2.564949,1.609438,2.397895,621626,2009-10-03,DH really enjoyed this dish. I did leave out t...,5,this dish is loaded with texture and flavor. w...
121672,279553,soup,turkey and brown rice chilli,14,485,202.4,2.0,37.0,22.0,34.0,...,3.610918,3.091042,3.526361,0.000000,2.302585,621626,2009-10-03,I made this on the stove top since it was a la...,5,fantstic crock pot recipe fron betty crocker. ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173921,23113,pasta,pumpkin risotto,12,30,405.6,16.0,8.0,9.0,22.0,...,2.079442,2.197225,3.091042,3.135494,3.044522,198154,2018-12-11,I made this as a side because DH is a meat and...,4,my ultimate comfort food. i make this so often...
173923,242547,meat,pulled chicken sandwiches crock pot,12,370,207.4,4.0,81.0,35.0,40.0,...,4.394449,3.555348,3.688879,0.693147,2.079442,218535,2018-12-16,amazing. one of the best crockpot bbq recipes,5,"a personal take on a recipe from the ""slow coo..."
173924,108068,pasta,velveeta macaroni and cheese,6,55,348.1,27.0,16.0,30.0,26.0,...,2.772589,3.401197,3.258097,3.761200,2.397895,218535,2018-12-16,I did not put the cheese slices on top. Still ...,5,"my son is a very picky eater, but loves velvee..."
173925,167834,vegetables,bacon wrapped asparagus,2,25,24.0,2.0,2.0,1.0,2.0,...,0.693147,0.000000,0.693147,1.098612,0.000000,218535,2018-12-16,so delicious that it's gone in the blink of an...,5,easy-to-prepare appetizer. laurita has a good...


## Remove new recipes from test data that are not in train data

In [156]:
print('train set: ', train_set.shape)
print('test set: ', test_set.shape)

train set:  (121668, 24)
test set:  (48955, 24)


In [157]:
train_recipe = train_set['recipe_id'].unique()
test_recipe = test_set['recipe_id'].unique()

print('train recipe count', len(train_recipe))
print('test recipe count: ', len(test_recipe))

train recipe count 64835
test recipe count:  33434


In [158]:
test_recipe = [x for x in test_recipe if x in train_recipe]
print(len(test_recipe))

12661


In [159]:
test_data = test_set[test_set['recipe_id'].isin(test_recipe)]
test_data.shape

(21893, 24)

## Number of users in train and test dataset

In [160]:
len(train_set["user_id"].unique())

378

In [118]:
len(test_data["user_id"].unique())

320

In [119]:
# convert to csv
train_set.to_csv('./Dataset/train_dataset_all.csv', index=False)
test_data.to_csv('./Dataset/test_dataset_all.csv', index=False)